### RatNDay3 Two novel info

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import subjects
from neuropy import analyses
from neuropy import plotting

sess = subjects.tn.ratNday3[0]
print(sess.recinfo)


### Set probe configuration
This animal had 8 shank probe from diagnostic biochips, copied from ratDay2NSD

In [ ]:
from neuropy.core import Shank, Probe, ProbeGroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
for i in range(8):

    shank = Shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=20,
        y_shift_per_column=[0, -15],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shanks.append(shank)
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)

probe1 = Probe(shanks)

prbgrp = ProbeGroup()
prbgrp.add_probe(probe1)

prbgrp.save(sess.filePrefix.with_suffix(".probegroup.npy"))
plot_probe(prbgrp)


In [ ]:
from neuropy.io import SpykingCircusIO

SpykingCircusIO.write_probe_file(
    sess.filePrefix.with_suffix(".prb"),
    sess.probegroup,
    rmv_badchans=True,
    combine_shanks=True,
)


### Artifacts epochs

In [ ]:
signal = sess.eegfile.get_signal([77])
artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=10, edge_cutoff=2)

metadata = artifact_epochs.metadata
artifact_epochs = artifact_epochs.from_array(
    artifact_epochs.starts - 0.1, artifact_epochs.stops, "artifact"
)
artifact_epochs.metadata = metadata

artifact_epochs.save(sess.filePrefix.with_suffix(".artifact.npy"))


In [ ]:
signal = sess.eegfile.get_signal([77])
plotting.plot_artifact_epochs(artifact_epochs, signal)


In [ ]:
from neuropy.io import SpykingCircusIO

file = sess.filePrefix.with_suffix(".dead")
SpykingCircusIO.write_epochs(file, sess.artifact)


In [ ]:
sess.recinfo.write_epochs(sess.artifact, "art")
